In [139]:
import json
import numpy as np
from pprint import pprint
from os import listdir
from os.path import isfile, join
from MongoOps import MongoOps
from ICSParser import ICSParser
from make_directions_matrix import DirectionsMatrix
import random

In [140]:
locations = []
mops = MongoOps()
sample_data_files = [f for f in listdir("/data") if isfile(join("/data", f))]
for open_house_file in sample_data_files:
    parser = ICSParser("/data/%s" % open_house_file)
    event = parser.to_dict()
    # print(event)
    result = mops.safe_query_for_location_info(event)
    locations += [result]

In [145]:
def random_combination(iterable, r):
    "Random selection from itertools.combinations(iterable, r)"
    pool = tuple(iterable)
    n = len(pool)
    indices = sorted(random.sample(range(n), r))
    return tuple(pool[i] for i in indices)

random_locations = random_combination(locations, 5)

In [146]:
DM = DirectionsMatrix(random_locations, mops)

In [147]:
DM.generate_simplified_directions_matrix()

In [148]:
sdm = DM.simplified_directions_matrix

In [149]:
# Showing off EST/EDT time of day of the open houses and the conversion to minutes from midnight that day
for i in range(len(sdm)):
    start = int(sdm[i]['dtstart'][9:-3])-600
    start_minutes = 6./10*start
    end = int(sdm[i]['dtend'][9:-3])-600
    end_minutes = 6./10*end
    print('%d (%f), %d (%f)' % (start, start_minutes, end, end_minutes))
    start = sdm[i]['start'] = start
    start_minutes = sdm[i]['start_minutes'] = start_minutes
    end = sdm[i]['end'] = end
    end_minutes = sdm[i]['end_minutes'] = end_minutes

900 (540.000000), 1100 (660.000000)
1000 (600.000000), 1200 (720.000000)
1000 (600.000000), 1200 (720.000000)
1100 (660.000000), 1300 (780.000000)
1200 (720.000000), 1400 (840.000000)


In [150]:
vertices = []
V = None
for i in range(len(sdm)):
    V = {'ID' : i, 
               'start' : sdm[i]['start_minutes'], 
               'end' : sdm[i]['end_minutes'], 
               'edges' : sdm[i]['durations'],
               'address_hash' : sdm[i]['address_hash']}
    vertices += [V]

In [151]:
def been_visited(visited, v):
    '''
    Checks to make sure you're not going back to a node that has already been visited.
    '''
    return v in visited

In [152]:
""" 
A Python Class
A simple Python graph class, demonstrating the essential 
facts and functionalities of graphs.
Original implementation from https://www.python-course.eu/graphs_python.php
Changes to include weighted edges from https://towardsdatascience.com/to-all-data-scientists-the-one-graph-algorithm-you-need-to-know-59178dbb1ec2
Some functions have been removed because they are not 
going to be used, and I would like to protect future 
users from using this graph object incorrectly.
"""

class Graph(object):
    def __init__(self, graph_dict=None):
        """ initializes a graph object 
            If no dictionary or None is given, 
            an empty dictionary will be used
        """
        if graph_dict == None:
            graph_dict = {}
        self.__graph_dict = graph_dict
        self.__vertices = self.vertices()

    def vertex_ids(self):
        """ returns the vertices of a graph """
        vertices = []
        for val in self.__graph_dict:
            vertices += [val['ID']]
        self.__vertices = vertices
        return list(self.__vertices)
    
    def vertices(self):
        """ returns the vertices of a graph """
        vertices = []
        for val in self.__graph_dict:
            ver = self.__removekey(val, 'edges')
            vertices += [ver]
        self.__vertices = vertices
        return list(self.__vertices)
    
    def get_vertex_from_vid(self, vid):
        ''' 
        returns the vertex given its id.
        I would LIKE to assume that the vertex's id will match its location in
        graph_dict, but I won't in the case that someone passes in an irregualar
        graph_dict to the Graph object.
        '''
        vertex = None
        for val in self.__graph_dict:
            if val['ID'] == vid:
                vertex = val
        if vertex == None:
            raise Exception('Vertex ID provided not found.')
        return vertex

    def edges(self):
        """ returns the edges of a graph """
        return self.__generate_edges()
    
    def get_edges_from_vid(self, vid):
        ''' 
        returns the edges from a vertex, given its id.
        '''
        vertex = self.get_vertex_from_vid(vid)
        edges = self.get_edges(vertex)
        return edges

    def __generate_edges(self):
        """ A static method generating the edges of the 
            graph "graph". Edges are represented as sets 
            with one (a loop back to the vertex) or two 
            vertices 
        """
        edges = []
        for vertex in self.__graph_dict:
            edges += self.get_edges(vertex)
        return edges

    def get_edges(self, vertex):
        edges = []
        for neighbour in vertex['edges']:
            weight = neighbour[1]
            neighbour = neighbour[0]
            v = vertex['ID']
            if (neighbour, vertex, weight) not in edges:
                edges.append([v, neighbour, weight])
        return edges
    
    def __str__(self):
        res = "vertices:\n"
        for k in self.__vertices:
            res += str(k) + " \n"
        res += "\nedges:\n"
        for edge in self.__generate_edges():
            res += str(edge) + " "
        return res

    def adj_mat(self):
        return self.__graph_dict
    
    def __removekey(self, d, key):
        r = dict(d)
        del r[key]
        return r

In [153]:
graph = Graph(vertices)
print(graph)

vertices:
{'ID': 0, 'start': 540.0, 'end': 660.0, 'address_hash': b'\x08\xfa{s@-G\xa0N5\xbf:e\xfc(\x004\x82\xa7#'} 
{'ID': 1, 'start': 600.0, 'end': 720.0, 'address_hash': b'\x7fn\xf1\x15\x03\x81\xd5\xc3\x06\x11\x133\xdd1#\x1f\xbe\x84y\xce'} 
{'ID': 2, 'start': 600.0, 'end': 720.0, 'address_hash': b'\xe2s\xe0\x1aA\xd7;\xd2-\x05\xb4}{\x8d\xaa\tK\xfe\x06t'} 
{'ID': 3, 'start': 660.0, 'end': 780.0, 'address_hash': b'\xaa\x84qy\x08\x8f\x18)\xf1\x8b3\xa2R\xbde\xc2M6N\x87'} 
{'ID': 4, 'start': 720.0, 'end': 840.0, 'address_hash': b'F\xeb\x03\xfe\xad\t\x9a\x8e\xae\x0fv\xaeV\xe9\xbd@\tkV\x9f'} 

edges:
[0, 1, 22.74] [0, 2, 19.05] [0, 3, 12.05] [0, 4, 16.33] [1, 0, 20.27] [1, 2, 8.12] [1, 3, 18.21] [1, 4, 11.42] [2, 0, 17.13] [2, 1, 8.62] [2, 3, 14.31] [2, 4, 8.4] [3, 0, 11.96] [3, 1, 19.19] [3, 2, 14.25] [3, 4, 16.32] [4, 0, 15.3] [4, 1, 11.93] [4, 2, 8.37] [4, 3, 17.16] 


In [155]:
print(graph.get_edges_from_vid(0))

[[0, 1, 22.74], [0, 2, 19.05], [0, 3, 12.05], [0, 4, 16.33]]


In [156]:
'''
For the sake of initial testing, I am going to start by going to the first house on the list (ID = 0).
Also, the starting time will be that starting time of the vertex we're starting with.
    A better starting time can be defined later (i.e. when routing is solved).
'''
starting_id = 0
starting_vertex = graph.get_vertex_from_vid(starting_id)
start_time = starting_vertex['start']

In [157]:
def visit_next(graph, current_vertex, arrival_time, average_time_at_each_house = 30, visited = []):
    idx = current_vertex['ID']
    print('Arrived at vertex %d at time %f' % (idx, arrival_time))
    visited += [idx]
    arrival_time += average_time_at_each_house
    
    print('Leaving at vertex %d at time %f' % (idx, arrival_time))
    
    edges = graph.get_edges(current_vertex)
    edges_no_cycle = get_acyclic_edges(edges, visited)
    
    print(visited, edges_no_cycle)
    
    if len(edges_no_cycle) > 0:
        selected_edge = edges_no_cycle[0]
        next_vertex = graph.get_vertex_from_vid(selected_edge[1])
        arrival_time += selected_edge[2]
        return visit_next(graph, next_vertex, arrival_time, visited = visited)
    else:
        hours_minutes = str(arrival_time / 60).split('.')
        hours_minutes[0] = hours_minutes[0]
        hours_minutes[1] = str(float('0.'+hours_minutes[1]) * .6)[2:4]
        
        print(hours_minutes)
        return visited, '%s:%s' %(hours_minutes[0], hours_minutes[1])
    

def get_acyclic_edges(edges, visited):
    E = []
    for edge in edges:
        idx = edge[1]
        if been_visited(visited, idx):
            pass
        else:
            E += [edge]
    return E
        
def been_visited(visited, v):
    '''
    Checks to make sure you're not going back to a node that has already been visited.
    '''
    return v in visited

In [158]:
visit_next(graph, starting_vertex, start_time)

Arrived at vertex 0 at time 540.000000
Leaving at vertex 0 at time 570.000000
[0] [[0, 1, 22.74], [0, 2, 19.05], [0, 3, 12.05], [0, 4, 16.33]]
Arrived at vertex 1 at time 592.740000
Leaving at vertex 1 at time 622.740000
[0, 1] [[1, 2, 8.12], [1, 3, 18.21], [1, 4, 11.42]]
Arrived at vertex 2 at time 630.860000
Leaving at vertex 2 at time 660.860000
[0, 1, 2] [[2, 3, 14.31], [2, 4, 8.4]]
Arrived at vertex 3 at time 675.170000
Leaving at vertex 3 at time 705.170000
[0, 1, 2, 3] [[3, 4, 16.32]]
Arrived at vertex 4 at time 721.490000
Leaving at vertex 4 at time 751.490000
[0, 1, 2, 3, 4] []
['12', '31']


([0, 1, 2, 3, 4], '12:31')